In [ ]:
import numpy as np
import pandas as pd
import glob, os


# Usage

Just select the correct pattern of the file/folders that you want to find and then write the output in a file for using with bash

### Example
find all the HorizonDump.h5 and output the file paths with a space in between. 

JoinH5 -o combined.h5 <paste_the_file_content>


# Code

In [ ]:
folder_path = "/panfs/ds09/sxs/himanshu/gauge_stuff/debug_gauge/runs_with_different_scale/1804_sph_kerr_deriv_wrt_inertial_25_50_50_amp_0.1"

# # With ringdowns
# lev_golb=folder_path+"/Ev/Lev**"

# Without ringdowns
lev_golb=folder_path+"/Ev/Lev1_??"



checkpoints_glob=lev_golb+"/Run/Checkpoints/?**"
checkpoints_A0_glob=lev_golb+"/Run/Checkpoints/?**/Cp-VarsGr_SphereA0**.h5"

horizon_dump_glob=lev_golb+"/Run/ApparentHorizons/HorizonsDump.h5"

pvd_files = lev_golb+"/Run/GaugeVis.pvd"




In [ ]:

path_pattern = horizon_dump_glob
path_collection = []


for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

In [ ]:
with open("./find_paths.ouput",'w') as f:
  for path in path_collection:
    f.write(path+" ")

if "Dump.h5" in path_pattern:
  with open("./find_paths.ouput",'w') as f:
    f.write("JoinH5 -o combined_horizondump.h5 ")
    for path in path_collection:
      f.write(path+" ")

In [ ]:

path_pattern = checkpoints_A0_glob
path_collection = []

def find_time(file_path):
    """" Reads time from Cp-EvolutionLoopControl.txt"""
    with open(file_path,'r') as f:
        a = f.readline()
        return float(a.split(";")[0].split("=")[1])
     


for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        time = find_time(folder_name[:-21]+"Cp-EvolutionLoopControl.txt")
        time_and_path = [time,folder_name]
        path_collection.append(time_and_path)
        # print(time_and_path)


time_sorted_path_list = list(pd.DataFrame(path_collection,columns=["Time","Path"]).sort_values(by=['Time'])['Path'])
time_sorted_path_list


with open("./find_paths.output",'w') as f:
  for path in time_sorted_path_list:
    f.write(path+" ")

## Combine pvd files

In [ ]:
def read_pvd_file(file_path):
  with open(file_path) as f:
    data = f.readlines()
    write_full_vtk_file_path(data,file_path)
    return data


def write_full_vtk_file_path(read_data,file_path):
  vtk_folder_name= file_path.split("/")[-1][:-4]
  vtk_folder_path= file_path[:-4]
  for i,path in enumerate(read_data):
    read_data[i] = path.replace(vtk_folder_name,vtk_folder_path)


def add_lists_skip_random(a,b):
  for i in b:
    if "timestep" in i:
      a.append(i)

In [ ]:
combined_data = read_pvd_file(path_collection[0])[:4]
for i in range(0,len(path_collection)):
  add_lists_skip_random(combined_data, read_pvd_file(path_collection[i])[3:-2]) #remove the last two lines and first three lines

combined_data = combined_data + read_pvd_file(path_collection[0])[-2:]


with open(folder_path+"/combined.pvd",'w') as f:
  f.writelines(combined_data)